In [ ]:
import ryn

In [ ]:
text_encoder_name = 'bert-base-cased'
dataset = 'oke.fb15k237_30061990_50'
date = '2020.09.29.180047'
run = f'distmult.{text_encoder_name}.{date}'

glob = (ryn.ENV.TEXT_DIR / 'mapper' / dataset / run).glob('weights/ryn/*/checkpoints/*')
available = {int(p.name[6:].split('.')[0]): p for p in glob}

print('checkpoints available for epochs: ', list(available))

In [ ]:
EPOCH = 7
model_path = available[EPOCH]

In [ ]:
# load config
wandb_id = model_path.parts[-3]

# they change the date format from YYYY.mm.dd.HHMMSS to YYYYmmdd_HHMMSS for whatever reasons
wandb_date = ''.join(date.split('.', maxsplit=2)).replace('.', '_')

config_path = (
    model_path.parents[4] / 'wandb' /
    f'run-{wandb_date}-{wandb_id}' / 'files' / 'config.yaml')

print(config_path)

import yaml
with config_path.open(mode='rb') as fd:
    config = yaml.load(fd, Loader=yaml.FullLoader)

print('loaded config')

In [ ]:
from ryn.text import mapper

mapper_config = mapper.MapperConfig(**config['mapper_config']['value'])
model = mapper.Mapper.from_config(config=mapper_config, text_encoder_name=text_encoder_name)

In [ ]:
# load weights
import torch
weights = torch.load(model_path)

model.load_state_dict(weights['state_dict'])
model.eval()
print('loaded model')

In [ ]:
model(
    sentences=torch.Tensor([(101, 11696, 1106, 9235, 1103, 2037, 5097, 1104, 103, 118, 2272, 4237, 119, 102), ]).to(dtype=torch.long, device=model.device),
    entities=torch.Tensor([(0, )]).to(dtype=torch.long, device=model.device))